In [56]:
#import all the dependencies
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, AveragePooling2D
from keras.layers import Input, concatenate
import os, time

In [57]:
# Preparing the dataset
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

In [58]:
CLASS_NAMES = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

# Preparing the dataset
X_val, y_val = train_images[:5000], train_labels[:5000]
X_train, y_train = train_images[5000:], train_labels[5000:]
X_test, y_test = test_images[500:], test_labels[500:]

# Building tensorflow datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()

print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

Training data size: 45000
Test data size: 9500
Validation data size: 5000


In [59]:
# AlexNet Model Code
def alexnet_process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 28x28 to 227x227
    image = tf.image.resize(image, (227,227))
    return image, label

alexnet_train_ds = (train_ds.map(alexnet_process_images).shuffle(buffer_size=train_ds_size).batch(batch_size=32, drop_remainder=True))
alexnet_test_ds = (test_ds.map(alexnet_process_images).shuffle(buffer_size=test_ds_size).batch(batch_size=32, drop_remainder=True))
alexnet_validation_ds = (validation_ds.map(alexnet_process_images).shuffle(buffer_size=validation_ds_size).batch(batch_size=32, drop_remainder=True))

print("AlexNet data size:", alexnet_train_ds)
print("AlexNet data size:", alexnet_test_ds)
print("AlexNet data size:", alexnet_validation_ds)


AlexNet data size: <BatchDataset element_spec=(TensorSpec(shape=(32, 227, 227, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 1), dtype=tf.uint8, name=None))>
AlexNet data size: <BatchDataset element_spec=(TensorSpec(shape=(32, 227, 227, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 1), dtype=tf.uint8, name=None))>
AlexNet data size: <BatchDataset element_spec=(TensorSpec(shape=(32, 227, 227, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 1), dtype=tf.uint8, name=None))>


In [60]:
def AlexNet():
    # Sequential
    keras.Sequential()

    # input layer 
    input_layer = Input(shape = (227, 227, 3))

    # convolutional layer: filters = 96, kernel_size = (11,11), strides = 4
    X = Conv2D(filters = 96, kernel_size = (11,11), strides = 4, padding = 'same', activation = 'relu')(input_layer)
	
	# BatchNormalization()
    X = BatchNormalization()(X)

    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

    # convolutional layer: filters = 256, kernel_size = (5,5), strides = 1
    X = Conv2D(filters = 256, kernel_size = (5,5), strides = 1, padding = 'same', activation = 'relu')(X)
	
	# BatchNormalization()
    X = BatchNormalization()(X)

    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)
	
	# convolutional layer: filters = 384, kernel_size = (3,3), strides = 1
    X = Conv2D(filters = 384, kernel_size = (3,3), strides = 1, padding = 'same', activation = 'relu')(X)
	
	# BatchNormalization()
    X = BatchNormalization()(X)
	
	# convolutional layer: filters = 384, kernel_size = (3,3), strides = 1
    X = Conv2D(filters = 384, kernel_size = (3,3), strides = 1, padding = 'same', activation = 'relu')(X)
	
	# BatchNormalization()
    X = BatchNormalization()(X)
	
	# convolutional layer: filters = 256, kernel_size = (3,3), strides = 1
    X = Conv2D(filters = 256, kernel_size = (3,3), strides = 1, padding = 'same', activation = 'relu')(X)
	
	# BatchNormalization()
    X = BatchNormalization()(X)

    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

    # Extra network:
    X = Flatten()(X)
    X = Dense(4096, activation = 'relu')(X)

    # Dropoutlayer 
    X = Dropout(0.5)(X)

    # output layer 
    X = Dense(1000, activation = 'softmax')(X)
    
    return X

In [61]:
# GoogleNet Model Code
def googlenet_process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 28x28 to 224x224
    image = tf.image.resize(image, (224,224))
    return image, label

googlenet_train_ds = (train_ds.map(googlenet_process_images).shuffle(buffer_size=train_ds_size).batch(batch_size=32, drop_remainder=True))
googlenet_test_ds = (test_ds.map(googlenet_process_images).shuffle(buffer_size=test_ds_size).batch(batch_size=32, drop_remainder=True))
googlenet_validation_ds = (validation_ds.map(googlenet_process_images).shuffle(buffer_size=validation_ds_size).batch(batch_size=32, drop_remainder=True))

print("GoogleNet data size:", googlenet_train_ds)
print("GoogleNet data size:", googlenet_test_ds)
print("GoogleNet data size:", googlenet_validation_ds)

GoogleNet data size: <BatchDataset element_spec=(TensorSpec(shape=(32, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 1), dtype=tf.uint8, name=None))>
GoogleNet data size: <BatchDataset element_spec=(TensorSpec(shape=(32, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 1), dtype=tf.uint8, name=None))>
GoogleNet data size: <BatchDataset element_spec=(TensorSpec(shape=(32, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 1), dtype=tf.uint8, name=None))>


In [62]:
def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4):
    # Input: 
    # - f1: number of filters of the 1x1 convolutional layer in the first path
    # - f2_conv1, f2_conv3 are number of filters corresponding to the 1x1 and 3x3 convolutional layers in the second path
    # - f3_conv1, f3_conv5 are the number of filters corresponding to the 1x1 and 5x5  convolutional layer in the third path
    # - f4: number of filters of the 1x1 convolutional layer in the fourth path

    # 1st path:
    path1 = Conv2D(filters = f1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)

    # 2nd path
    path2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
    path2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

    # 3rd path
    path3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
    path3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

    # 4th path
    path4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
    path4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

    output_layer = concatenate([path1, path2, path3, path4], axis = 3)

    return output_layer

In [63]:
def GoogLeNet():
    # input layer 
    input_layer = Input(shape = (224, 224, 3))

    # convolutional layer: filters = 64, kernel_size = (7,7), strides = 2
    X = Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'same', activation = 'relu')(input_layer)

    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

    # convolutional layer: filters = 64, strides = 1
    X = Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)

    # convolutional layer: filters = 192, kernel_size = (3,3)
    X = Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)

    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

    # 1st Inception block
    X = Inception_block(X, f1 = 64, f2_conv1 = 96, f2_conv3 = 128, f3_conv1 = 16, f3_conv5 = 32, f4 = 32)

    # 2nd Inception block
    X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 192, f3_conv1 = 32, f3_conv5 = 96, f4 = 64)

    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

    # 3rd Inception block
    X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 208, f3_conv1 = 16, f3_conv5 = 48, f4 = 64)

    # Extra network 1:
    X1 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
    X1 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
    X1 = Flatten()(X1)
    X1 = Dense(1024, activation = 'relu')(X1)
    X1 = Dropout(0.7)(X1)
    X1 = Dense(5, activation = 'softmax')(X1)

    # 4th Inception block
    X = Inception_block(X, f1 = 160, f2_conv1 = 112, f2_conv3 = 224, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

    # 5th Inception block
    X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 256, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

    # 6th Inception block
    X = Inception_block(X, f1 = 112, f2_conv1 = 144, f2_conv3 = 288, f3_conv1 = 32, f3_conv5 = 64, f4 = 64)

    # Extra network 2:
    X2 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
    X2 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
    X2 = Flatten()(X2)
    X2 = Dense(1024, activation = 'relu')(X2)
    X2 = Dropout(0.7)(X2)
    X2 = Dense(1000, activation = 'softmax')(X2)

    # 7th Inception block
    X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128)

    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

    # 8th Inception block
    X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128)

    # 9th Inception block
    X = Inception_block(X, f1 = 384, f2_conv1 = 192, f2_conv3 = 384, f3_conv1 = 48, f3_conv5 = 128, f4 = 128)

    # Global Average pooling layer 
    X = GlobalAveragePooling2D(name = 'GAPL')(X)

    # Dropoutlayer 
    X = Dropout(0.4)(X)

    # output layer 
    X = Dense(1000, activation = 'softmax')(X)

    return X

In [64]:
AlexNetInput = AlexNet()
print("AlexNet Model:", AlexNetInput)

GoogleNetInput = GoogLeNet()
print("GoogleNet Model:", GoogleNetInput)

ModelInputLayer = concatenate([AlexNetInput, GoogleNetInput], axis = 1)
print("Input Model:", ModelInputLayer)

AlexNet Model: KerasTensor(type_spec=TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None), name='dense_36/Softmax:0', description="created by layer 'dense_36'")
GoogleNet Model: KerasTensor(type_spec=TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None), name='dense_41/Softmax:0', description="created by layer 'dense_41'")
Input Model: KerasTensor(type_spec=TensorSpec(shape=(None, 2000), dtype=tf.float32, name=None), name='concatenate_59/concat:0', description="created by layer 'concatenate_59'")


In [65]:
# input layer 
input_layer = Input(ModelInputLayer)
	
# output layer 
ModelOutputLayer = Dense(10, activation = 'softmax')(input_layer)

# model
model = Model(inputs = input_layer, outputs = [ModelOutputLayer], name = 'SVMModel')
model.summary()

TypeError: ignored